In [ ]:
import json
from langchain_community.vectorstores import FAISS
from langchain.embeddings.base import Embeddings
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
import numpy as np
import hashlib
from typing import List
import graphviz
from pathlib import Path
import datetime

In [ ]:
# Define our embedding class (same as before)
class SimpleHashEmbeddings(Embeddings):
    def __init__(self, dimension: int = 384):
        self.dimension = dimension
    
    def _hash_text(self, text: str) -> List[float]:
        hash_object = hashlib.sha256(text.encode())
        hash_hex = hash_object.hexdigest()
        float_array = []
        for i in range(0, len(hash_hex), 8):
            chunk = hash_hex[i:i+8]
            float_val = int(chunk, 16) / 2**32 - 1
            float_array.append(float_val)
        array = np.array(float_array, dtype=np.float32)
        if len(array) < self.dimension:
            array = np.pad(array, (0, self.dimension - len(array)))
        else:
            array = array[:self.dimension]
        norm = np.linalg.norm(array)
        if norm > 0:
            array = array / norm
        return array.tolist()
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self._hash_text(text) for text in texts]
    
    def embed_query(self, text: str) -> List[float]:
        return self._hash_text(text)

In [ ]:
# Define our documentation prompts
documentation_prompts = {
    'overview': PromptTemplate(
        input_variables=["code"],
        template="""Analyze the following PL/SQL chess engine code and provide a comprehensive overview:

{code}

Please provide:
1. High-level overview of the system
2. Main components and their purposes
3. Key functionalities
4. Design patterns used
5. Important dependencies
"""
    ),
    
    'architecture': PromptTemplate(
        input_variables=["code"],
        template="""Analyze the following PL/SQL chess engine code and describe its architecture:

{code}

Please provide:
1. System architecture overview
2. Component interactions
3. Data flow description
4. Key interfaces
5. Architectural patterns used

Also provide a mermaid diagram showing the architecture.
"""
    ),
    
    'procedure_analysis': PromptTemplate(
        input_variables=["code", "procedure_name"],
        template="""Analyze the following PL/SQL procedure/function in detail:

Procedure/Function: {procedure_name}

Code:
{code}

Please provide:
1. Purpose and functionality
2. Input parameters and their usage
3. Return values and their meaning
4. Algorithm explanation
5. Error handling approach
6. Performance considerations
7. Dependencies and side effects
"""
    ),
    
    'migration_analysis': PromptTemplate(
        input_variables=["code"],
        template="""Analyze the following PL/SQL code and provide a detailed migration approach:

{code}

Please provide:
1. Recommended target technology stack
2. Step-by-step migration plan
3. Potential challenges and solutions
4. Required code transformations
5. Testing strategy
6. Estimated effort and complexity
"""
    )
}

In [ ]:
# Load vector store and metadata
print("Loading vector store and metadata...")
embeddings = SimpleHashEmbeddings(dimension=384)
vectorstore = FAISS.load_local("vectorstore", embeddings)

with open("chess_metadata.json", "r") as f:
    metadata = json.load(f)

# Initialize Ollama
llm = Ollama(
    model="llama2:latest",
    temperature=0.2,
    verbose=True,
    base_url="http://localhost:11434"
)

# Create documentation chains
doc_chains = {name: LLMChain(llm=llm, prompt=prompt) 
             for name, prompt in documentation_prompts.items()}

In [ ]:
def generate_full_documentation():
    """Generate comprehensive documentation for the chess engine"""
    documentation = {
        'generated_at': datetime.datetime.now().isoformat(),
        'overview': {},
        'architecture': {},
        'packages': {},
        'migration_analysis': {}
    }
    
    print("Generating system overview...")
    # Get overall system context
    system_docs = vectorstore.similarity_search(
        "chess engine system overview architecture main components", k=5)
    combined_system_text = "\n\n".join([doc.page_content for doc in system_docs])
    
    # Generate overview
    documentation['overview'] = doc_chains['overview'].run(code=combined_system_text)
    
    # Generate architecture documentation
    documentation['architecture'] = doc_chains['architecture'].run(code=combined_system_text)
    
    print("\nAnalyzing individual packages...")
    # Process each package
    for package in metadata['packages']:
        package_name = package['package_name']
        print(f"Processing package: {package_name}")
        
        package_docs = vectorstore.similarity_search(
            f"package {package_name} implementation details", k=3)
        package_text = "\n\n".join([doc.page_content for doc in package_docs])
        
        package_info = {
            'overview': doc_chains['overview'].run(code=package_text),
            'procedures': {}
        }
        
        # Process each procedure in the package
        for routine in package['routines']:
            routine_name = routine['name']
            print(f"  Analyzing procedure: {routine_name}")
            
            routine_docs = vectorstore.similarity_search(
                f"package {package_name} procedure {routine_name} implementation", k=2)
            routine_text = "\n\n".join([doc.page_content for doc in routine_docs])
            
            package_info['procedures'][routine_name] = \
                doc_chains['procedure_analysis'].run(
                    code=routine_text,
                    procedure_name=routine_name
                )
        
        documentation['packages'][package_name] = package_info
    
    print("\nGenerating migration analysis...")
    # Generate migration analysis
    documentation['migration_analysis'] = doc_chains['migration_analysis'].run(
        code=combined_system_text)
    
    # Save documentation
    output_file = f"chess_engine_documentation_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(output_file, 'w') as f:
        json.dump(documentation, f, indent=2)
    
    print(f"\nDocumentation generated and saved to {output_file}")
    return documentation

In [ ]:
# Generate markdown documentation
def generate_markdown_documentation(documentation):
    """Convert the JSON documentation to markdown format"""
    md_content = []
    
    # Title
    md_content.append("# Chess Engine Documentation")
    md_content.append(f"Generated on: {documentation['generated_at']}\n")
    
    # System Overview
    md_content.append("## System Overview")
    md_content.append(documentation['overview'])
    
    # Architecture
    md_content.append("\n## System Architecture")
    md_content.append(documentation['architecture'])
    
    # Packages
    md_content.append("\n## Packages")
    for package_name, package_info in documentation['packages'].items():
        md_content.append(f"\n### Package: {package_name}")
        md_content.append("#### Overview")
        md_content.append(package_info['overview'])
        
        md_content.append("\n#### Procedures and Functions")
        for proc_name, proc_doc in package_info['procedures'].items():
            md_content.append(f"\n##### {proc_name}")
            md_content.append(proc_doc)
    
    # Migration Analysis
    md_content.append("\n## Migration Analysis")
    md_content.append(documentation['migration_analysis'])
    
    # Save to file
    output_file = f"chess_engine_documentation_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
    with open(output_file, 'w') as f:
        f.write('\n'.join(md_content))
    
    print(f"Markdown documentation saved to {output_file}")
    return output_file

In [ ]:
# Generate complete documentation
documentation = generate_full_documentation()

# Convert to markdown
markdown_file = generate_markdown_documentation(documentation)

print("\nDocumentation generation complete!")
print(f"JSON documentation and Markdown files have been generated.")